In [ ]:
import os, sys
projectDir = os.path.abspath('..')
if not projectDir in sys.path: sys.path.append(projectDir)
import matplotlib.pyplot as plt
import numpy as np
import scipy
import bspy

# The $n$-body equations are a 2nd order system of ODEs

$$m_ix_i'' = \sum_{j\ne i}f_{ij}(x_j-x_i)/r_{ij},\qquad i=1,...,n$$

where

$$f_{ij}={Gm_im_j\over r_{ij}^2},\qquad\hbox{with } r_{ij}=\|x_i-x_j\|$$

With the masses given in kilograms and the distances in kilometers, the appropriate value of $G$ is

$$G=8.6443\times 10^{-13}{\hbox{km}^3\over\hbox{hour}^2\hbox{kg}}$$

This can be rewritten in a simpler form

$$x_i'' = \sum_{j\ne i}g_{ij}(x_j-x_i),\qquad i = 1, ..., n$$

where

$$g_{ij}={Gm_j\over r_{ij}^3},\qquad\hbox{with } r_{ij}=\|x_i-x_j\|$$

In [ ]:
def nBodyF(t, u, nBody, nDim, mass):
    G = 8.6443e-13
    nState = nDim * nBody
    rhs = np.zeros((nState,))
    jacobian = np.zeros((nState, nState, 2))
    for iBody in range(nBody):
        bodySlice = slice(iBody * nDim, (iBody + 1) * nDim)
        for iTerm in range(nBody):
            if iBody == iTerm:
                continue
            termSlice = slice(iTerm * nDim, (iTerm + 1) * nDim)
            direction_ij = u[termSlice, 0] - u[bodySlice, 0]
            rij = (direction_ij @ direction_ij) ** 0.5
            gij = G * mass[iTerm] / rij ** 3
            gijPrime = 3.0 * gij / rij ** 2
            rhs[bodySlice] += gij * direction_ij
            jacobian[bodySlice, bodySlice, 0] -= gij * np.identity(nDim)
            jacobian[bodySlice, termSlice, 0] += gij * np.identity(nDim)
            jacobian[bodySlice, bodySlice, 0] += gijPrime * direction_ij * direction_ij
            jacobian[bodySlice, termSlice, 0] -= gijPrime * direction_ij * direction_ij
    return rhs, jacobian

In [ ]:
nBody = 2
nDim = 1
mass = np.array([3.0e+19, 1.0e+19])
finalTime = 4.0
mass = np.array([3.0e+19, 1.0e+19])
u = [0.0, 0.0, finalTime, finalTime]
data = np.array([[100.0, 900.0], [0.0, 0.0],
                 [900.0, 100.0], [0.0, 0.0]]).T
initialGuess = bspy.Spline.least_squares(u, data)
solution = initialGuess.solve_ode(2, 0, nBodyF, 1.0e-4, (nBody, nDim, mass))

In [ ]:
pValues = np.linspace(0.0, 4.0, 5)
tValues = np.linspace(pValues[0], pValues[-1], 101)
x1Points, x2Points = solution(pValues)
x1Values, x2Values = solution(tValues)
plt.plot(x1Points, np.zeros(len(x1Points)), 'b', marker = 'o', markersize = 8, linestyle = "")
plt.plot(x2Points, np.zeros(len(x2Points)), 'b', marker = 'o', markersize = 5, linestyle = "")
plt.plot(x1Values, np.zeros(len(x1Values)), 'g')
plt.plot(x2Values, np.zeros(len(x2Values)), 'g')
plt.axis('equal')

In [ ]:
nBody = 2
nDim = 2
mass = np.array([3.0e+19, 1.0e+19])
finalTime = 10.0
mass = np.array([3.0e+19, 1.0e+19])
u = [0.0, 0.0, finalTime, finalTime]
data = np.array([[100.0, 0.0, 900.0, 900.0], [0.0, 100.0, 0.0, -100.0],
                 [900.0, 100.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]).T
initialGuess = bspy.Spline.least_squares(u, data)
solution = initialGuess.solve_ode(2, 0, nBodyF, 1.0e-4, (nBody, nDim, mass))
print(solution(0.0), solution.derivative([1], [0.0]))

In [ ]:
pValues = np.linspace(0.0, finalTime, int(finalTime) + 1)
tValues = np.linspace(pValues[0], pValues[-1], 101)
x1P, y1P, x2P, y2P = solution(pValues)
x1, y1, x2, y2 = solution(tValues)
plt.plot(x1P, y1P, 'b', marker = 'o', markersize = 8, linestyle = "")
plt.plot(x2P, y2P, 'b', marker = 'o', markersize = 5, linestyle = "")
plt.plot(x1, y1, 'g')
plt.plot(x2, y2, 'g')
plt.axis('equal')

In [ ]:
finalTime += 5.0
initialGuess = solution.extrapolate([[None, finalTime]], 2).remove_knots(0.01, 2, 0)
solution = initialGuess.solve_ode(2, 0, nBodyF, 1.0e-4, (nBody, nDim, mass))
pValues = np.linspace(0.0, finalTime, 11)
tValues = np.linspace(pValues[0], pValues[-1], 201)
x1P, y1P, x2P, y2P = solution(pValues)
x1, y1, x2, y2 = solution(tValues)
plt.plot(x1P, y1P, 'b', marker = 'o', markersize = 8, linestyle = "")
plt.plot(x2P, y2P, 'b', marker = 'o', markersize = 5, linestyle = "")
plt.plot(x1, y1, 'g')
plt.plot(x2, y2, 'g')
plt.axis('equal')

In [ ]:
nBody = 3
nDim = 2
mass = np.array([3.0e+19, 1.0e+19, 2.0e+19])
finalTime = 3.0
u = [0.0, 0.0, finalTime, finalTime]
data = np.array([[100.0, 100.0, 100.0, 900.0, 900.0, 900.0], [150.0, 0.0, 0.0, -100.0, -40.0, 0.0],
                 [300.0, 100.0, 100.0, 700.0, 820.0, 900.0], [150.0, 0.0, 0.0, -100.0, -40.0, 0.0]]).T
initialGuess = bspy.Spline.least_squares(u, data)
solution = initialGuess.solve_ode(2, 0, nBodyF, 1.0e-4, (nBody, nDim, mass))
print(solution.derivative([1], [0.0]))

In [ ]:
pValues = np.linspace(0.0, finalTime, int(finalTime) + 1)
tValues = np.linspace(pValues[0], pValues[-1], 101)
x1P, y1P, x2P, y2P, x3P, y3P = solution(pValues)
x1, y1, x2, y2, x3, y3 = solution(tValues)
plt.plot(x1P, y1P, 'b', marker = 'o', markersize = 8, linestyle = "")
plt.plot(x2P, y2P, 'b', marker = 'o', markersize = 5, linestyle = "")
plt.plot(x3P, y3P, 'b', marker = 'o', markersize = 6.5, linestyle = "")
plt.plot(x1, y1, 'g')
plt.plot(x2, y2, 'g')
plt.plot(x3, y3, 'g')
plt.axis('equal')

In [ ]:
finalTime += 0.1
print(finalTime)
initialGuess = solution.extrapolate([[None, finalTime]], 2).remove_knots(0.01, 2, 0)
solution = initialGuess.solve_ode(2, 0, nBodyF, 1.0e-4, (nBody, nDim, mass))
pValues = np.linspace(0.0, finalTime, int(finalTime) + 1)
tValues = np.linspace(pValues[0], pValues[-1], 101)
x1P, y1P, x2P, y2P, x3P, y3P = solution(pValues)
x1, y1, x2, y2, x3, y3 = solution(tValues)
plt.plot(x1P, y1P, 'b', marker = 'o', markersize = 8, linestyle = "")
plt.plot(x2P, y2P, 'b', marker = 'o', markersize = 5, linestyle = "")
plt.plot(x3P, y3P, 'b', marker = 'o', markersize = 6.5, linestyle = "")
plt.plot(x1, y1, 'g')
plt.plot(x2, y2, 'g')
plt.plot(x3, y3, 'g')
plt.axis('equal')